In [1]:
import numpy as np
import pandas as pd
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn import preprocessing
from sklearn.metrics import roc_curve, auc
from ipywidgets import IntProgress
from tqdm import tqdm_notebook as tqdm

# silences package warnings
import warnings
warnings.filterwarnings('ignore')

In [2]:
def prepare_df(df, target_var, cont_vars=[], cat_vars=[]):
    total_vars = cont_vars + cat_vars + [target_var]
    model_df = df[total_vars]
    cleaned_df = model_df.dropna(subset=total_vars)

    # turns categorical variables into dummy variables
    for var in cat_vars:
        temp_dummy = pd.get_dummies(cleaned_df[var], drop_first=True)
        cleaned_df = pd.concat([cleaned_df.drop([var], axis=1), temp_dummy], axis=1)

    # normalize the data
    for var in cont_vars:
        cleaned_df[var] = preprocessing.scale(cleaned_df[var])

    return cleaned_df

In [3]:
def xgbclassify(df, model, params=None, trainCV='none', target='txgot_binary', folds=5, iterations=100, print_id=1):
    
    feat_vars = [var for var in list(df.columns) if var != target]
    X = df[feat_vars].values
    y = df[target].values
    
    if trainCV=='grid':
        param_bins = {key: {i:0 for i in params[key]} for key in params.keys()}
    
    avg_train_auc = 0
    avg_test_auc = 0
    avg_train_acc = 0
    avg_test_acc = 0
    
    rskf = RepeatedStratifiedKFold(n_splits=folds, n_repeats=iterations)
    for train_index, test_index in tqdm(rskf.split(X, y)):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        
        train_auc, test_auc, train_acc, test_acc, best_params = xgbmodel(model, params, 
                                                                         X_train, X_test, 
                                                                         y_train, y_test, 
                                                                         trainCV, print_id)
        
        if trainCV=='grid':
            for key, value in best_params.items():
                param_bins[key][value] += 1
        
        avg_train_auc += train_auc
        avg_test_auc += test_auc
        avg_train_acc += train_acc
        avg_test_acc += test_acc
        
    avg_train_auc /= (folds * iterations)
    avg_test_auc /= (folds * iterations)
    avg_train_acc /= (folds * iterations)
    avg_test_acc /= (folds * iterations)
    
    print('Train vs. Test')
    print('Training AUC: {}'.format(round(avg_train_auc, 3)))
    print('Testing AUC: {}'.format(round(avg_test_auc, 3)))
    print()
    print('Training Accuracy: {}'.format(round(avg_train_acc, 3)))
    print('Testing Accuracy: {}'.format(round(avg_test_acc, 3)))
    
    if trainCV=='grid':
        print()
        for key, value in param_bins.items():
            print('{}: {}'.format(key, value))
    
    print()
    print('Feature Importance:')
    if not (trainCV is 'random' or trainCV is 'grid'):
        sorted_idx = np.argsort(model.feature_importances_)[::-1]
        for index in sorted_idx:
            print([feat_vars[index], model.feature_importances_[index]])
        
    return avg_train_auc, avg_test_auc    

In [4]:
def xgbmodel(model, params, X_train, X_test, y_train, y_test, trainCV, print_id):
    if trainCV is 'random':
        skf = StratifiedKFold(n_splits=5, shuffle = True, random_state=print_id)
        
        X = np.concatenate((X_train, X_test), axis=0)
        Y = np.concatenate((y_train, y_test), axis=0)
        
        my_model = RandomizedSearchCV(model, param_distributions=params, n_iter=5, 
                           scoring='roc_auc', n_jobs=4, 
                           cv=skf.split(X, Y), verbose=3)
        
        my_model.fit(X, Y)
        best_params = my_model.best_params_
        # print(best_params)
        
    elif trainCV is 'grid':
        X = np.concatenate((X_train, X_test), axis=0)
        Y = np.concatenate((y_train, y_test), axis=0)

        skf = StratifiedKFold(n_splits=5, shuffle = True)
        
        my_model = GridSearchCV(model, params, scoring='roc_auc', cv=skf.split(X,Y))
                
        my_model.fit(X, Y)
        best_params = my_model.best_params_
        # print(best_params)
        
    else:
        my_model = model
        
        my_model.fit(X_train, y_train)

#         eval_set=[(X_train, y_train), (X_test, y_test)]
#         my_model.fit(X_train, y_train, eval_set=eval_set, 
#                      eval_metric="auc", early_stopping_rounds=15, verbose=False)
        
        best_params = params
    
    train_pred = my_model.predict(X_train)
    test_pred = my_model.predict(X_test)
    
    fpr, tpr, _ = roc_curve(y_train, train_pred, pos_label=1)
    train_auc_score = auc(fpr, tpr)
    
    fpr, tpr, _ = roc_curve(y_test, test_pred, pos_label=1)
    test_auc_score = auc(fpr, tpr)
    
    train_auc = train_auc_score
    test_auc = test_auc_score
    
    train_acc = my_model.score(X_train, y_train)
    test_acc = my_model.score(X_test, y_test)
    
    test_metrics = confusion_matrix(y_test, test_pred).ravel()
    train_metrics = confusion_matrix(y_train, train_pred).ravel()

    return train_auc, test_auc, train_acc, test_acc, best_params

In [6]:
topics_df = pd.read_csv('../../DataPlus/dvd_shared_slda_topics.csv')

In [7]:
list(topics_df.columns)

['Unnamed: 0',
 'Unnamed: 0.1',
 'Tx3',
 'Advice1',
 'Anx11',
 'Anx111',
 'Anx112',
 'Anx113',
 'Anx12',
 'Anx13',
 'Anx51',
 'Anx52',
 'Anx53',
 'Anx61',
 'Anx62',
 'Anx63',
 'Anx71',
 'Anx72',
 'Anx73',
 'Anx91',
 'Anx92',
 'Anx93',
 'Ask3',
 'Clear3',
 'DA1',
 'Dienot2',
 'Doc1',
 'Explain3',
 'Finaltx3',
 'ID',
 'Involve3',
 'MD_age',
 'MD_gender',
 'MD_number_pts_wk',
 'MD_percentpts',
 'MD_race',
 'MD_specialty',
 'MD_type',
 'MD_yrgrad',
 'Mdtxrec3',
 'Opinion3',
 'Raded2',
 'Radpee2',
 'Satis3',
 'Study',
 'Sured2',
 'Surpee2',
 'Talkda3',
 'Timeda2',
 'TxgotTx3cc',
 'Txlean3',
 'Wait2',
 'Whyww2',
 'Wwed2',
 'Wwpee2',
 'age',
 'arabme',
 'asian',
 'black',
 'education',
 'gleason',
 'hispanic',
 'irespmd',
 'marry',
 'mdrespme',
 'native',
 'pacific',
 'psa1',
 'raceother',
 'txgot',
 'txgot_binary',
 'white',
 'Convo_1',
 'Convo_2',
 'Doctor_1',
 'Doctor_2',
 'Topic1',
 'Topic2',
 'Topic3',
 'Topic4',
 'Topic5',
 'Topic6',
 'Topic7',
 'Topic8',
 'Topic9',
 'Topic10',
 'Topic1

In [24]:
cat1 = ['gleason', 'DVD_UroRec_AS', 'DVD_UroRec_AS', 'DVD_UroRec_AS']
cont1 = ['age', 'DVD_UroTalk_AS', 'DVD_UroTalk_surgery', 'DVD_UroTalk_rad']
subset1 = ['Topic1', 'Topic3', 'Topic8', 'Topic12', 'Topic13', 'Topic14']
subset2 = ['Topic4', 'Topic8', 'Topic12', 'Topic13', 'Topic14']
subset3 = ['Topic3', 'Topic4', 'Topic6', 'Topic8', 'Topic9', 'Topic14', 'Topic15']

In [25]:
first_df = prepare_df(topics_df, 'txgot_binary', subset1 + cont1, cat1)

KeyError: "['DVD_UroTalk_AS' 'DVD_UroTalk_surgery' 'DVD_UroTalk_rad' 'DVD_UroRec_AS'\n 'DVD_UroRec_AS' 'DVD_UroRec_AS'] not in index"

## Grid Search

In [22]:
grid_params = {
    'min_child_weight': [1, 2],
    'gamma': [0, 0.1, 0.2],
    'subsample': [0.5, 0.6],
    'colsample_bytree': [0.1, 0.2, 0.3],
    'max_depth': [2, 3, 4],
    'n_estimators': [150, 160]
}

In [23]:
xgb4 = XGBClassifier(learning_rate=0.02, objective='binary:logistic',
                    silent=True, nthread=1)
xgbclassify(first_df, xgb4, params=grid_params, 
            trainCV='grid', iterations=40, print_id=3)


Train vs. Test
Training AUC: 0.674
Testing AUC: 0.681

Training Accuracy: 0.975
Testing Accuracy: 0.974

min_child_weight: {1: 175, 2: 25}
gamma: {0: 78, 0.1: 63, 0.2: 59}
subsample: {0.5: 98, 0.6: 102}
colsample_bytree: {0.1: 70, 0.2: 56, 0.3: 74}
max_depth: {2: 74, 3: 66, 4: 60}
n_estimators: {150: 94, 160: 106}

Feature Importance:


(0.6744916666666663, 0.6810714285714288)